In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Importing the necessary libraries

In [ ]:
import nltk 
import random 
import logging 

import tensorflow as tf 
import keras 

nltk.download('punkt')
# chỉ ghi lại thông báo lỗi 
tf.get_logger().setLevel(logging.ERROR)

# Set random seed 
keras.utils.set_random_seed(42)

Define certain variables 

In [ ]:
TOKENIZER_BATCH_SIZE = 256 # KÍCH THƯỚC HÀNG LOẠT ĐỂ ĐÀO TẠO MÃ THÔNG BÁO TRÊN 
TOKENIZER_VOCABULARY = 25000 # TỔNG SỐ TỪ PHỤ DUY NHẤT MÀ TOKENIZER CÓ THỂ CÓ 

BLOCK_SIZE = 128 # SỐ LƯỢNG TỐI ĐA CỦA TOKEN TRONG MỖI MẪU ĐẦU VÀO 
NSP_PROB = 0.5  # XÁC XUẤT CÂU TIẾP THEO LÀ CÂU TIẾP THEO THỰC TẾ TRONG NSP 
SHORT_SEQ_PROB = 0.1 # Xác xuất tạo ra các chuỗi ngăn hơn để giảm thiểu 
# sự không khớp giữa tiền huấn luyện và tinh chỉnh 
MAX_LENGTH = 512 # Số lượng token tối đa trong mỗi đầu vào sau đệm

MLM_PROB = 0.2  # Xác xuất với những tokens là mặt nạ trong MLM 

TRAIN_BATCH_SIZE = 2 # Kích thước lô cho mô hình huấn luyện trước trên 
MAX_EPOCHS = 1 # SỐ KỶ NGUYÊN TỐI ĐA CHO HUẤN LUYỆN MÔ HÌNH 
LEARNING_RATE = 1e-4 # TỶ LỆ HỌC TẬP CHO HUẤN LUYỆN MÔ HÌNH 

MODEL_CHECKPOINT = 'bert-base-cased' # Mô hình huấn luyện trước từ 🤗 Model Hub

Load the WikiText dataset 

In [ ]:
from datasets import load_dataset 
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
print(dataset)

Training a new Tokenizer

In [ ]:
all_texts =  [
    _ for _ in dataset['train']['text'] if len(_) > 0 and not _.startswith("=")
]
# Xây dựng hàm để tách lô văn bản 
def batch_iterator(): 
    # lặp qua tất cả dữ liệu với mỗi bước nhảy bằng 1 lô
    for i in range(0 ,len(all_texts), TOKENIZER_BATCH_SIZE):
        # Sử dụng từ khóa yield để trả về 1 lô văn bản từ danh sách 
        # kích thước mỗi lô = TOKENIZER_BATCH_SIZE 
        yield all_texts[i : i + TOKENIZER_BATCH_SIZE]

In [ ]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator() , vocab_size=TOKENIZER_VOCABULARY 
)

Data Pre-processing 

In [ ]:
# lấy 1000 dữ liệu từ dữ liệu train gốc 
dataset['train'] = dataset['train'].select([_ for _ in range(1000)])
# Tương tự với validation data
dataset['validation'] = dataset['validation'.select([i for i in range(1000)])]

In [ ]:
# We difine the số lượng tokes tối đa sau khi thực hiện tokenization mỗi mẫu huấn luyện
# will have
max_num_tokens = BLOCK_SIZE - tokenizer.num_special_tokens_to_add(pair=True)
def prepare_train_features(examples):
    """Function to prepare features for NSP task
    
    Argument: 
        example: A dictionary with 1 key ('text')
        text : List of raw documents (str)
    Returns: 
        examples: A dictionary with 4 keys 
            Input_ids : List of Tokenized , Concatnated , and Bacthed (từng đợt)
                sentences form the individual (cá nhân) raw documents (int)
            Token_type_ids : List of intergers (0 or 1) corresponding 
                To : 0 for sentences no.1  and padding , 1 for sentence no.2 and padding 
            Attention_mask: List of intergers (0 or 1) corresponding 
                To : 1 for non-padded tokens , o for padded
            Next_sentence_label: List of intergers (0 or 1) corresponding
                To : 1 if the second sentence actually follows the fist  
                0 if the senetence is sampled from somewhere else in the corpus
    """

    # Xóa đi những mẫu không mong muốn từ dữ liệu huấn luyện 
    # Sử dụng hàm strip () để biến đổi các texts trong document thành string 
    # mặc định hàm này sẽ loại bỏ khoảng trắng 
    # sau đó kiểm tra xem chuỗi d có bắt đầu bằng ký tự "=" hay không 
    examples['document'] = [
        d.strip() for d in examples['document'] if len(d) > 0 and not d.startswitch(" =")
    ]
    # Tách các dữ liệu từ tập dữ liệu thành các câu riêng lẻ 
    # sử dụng hàm nltk.tokenize.sent_tokenize để trả về danh sách chuỗi 
    # mỗi chuỗi là một câu  
    examples['sentences'] = [
        nltk.tokenize.sent_tokenize(document) for document in examples['document']
    ]
    # Chuyển đổi các mã thông báo trong câu thành id (int) bằng cách sử dụng 
    # mã thông báo được đào tạo 
    examples['tokenized_sentences'] = [
        [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) for sent in doc ]
        for doc in examples['sentences']
    ]
    
    # xác đinh danh sách đầu ra 
    examples['input_ids'] = []  # id return  input tokens type int 
    examples['token_type_ids'] = []  # sentences [0 or 1] second and first 
    examples['attention_mask']  = []  # [o , 1] : no 1  , for sentences , 0 for padded 
    examples['next_sentence_labels']=[]   #[0,1]: 1 is follower , 0 is not followed

    # Duyệt qua tokens id của mẫu lấy ra vị trí và giá trị 
    # Sử dụng enumerate như một iterator một biến chứa giá trị có thể lặp 
    for doc_index , document in enumerate(examples['tokenized_sentences']):
        # khởi tạo 2 mảng = None 
        # một mảng sẽ chứa 1 bộ đệm lưu trữ các phân đoạn làm việc hiên tại 
        current_chunk = []
        # và một bộ sẽ chứa độ dài câu hiện tại 
        current_length = []
        i = 0 

        """ 
            Để có thể xử lý dễ dang hơn , thường tạo ra các đoạn văn bản có chiều dài 
                bằng nhau block_size , 
            Đôi khi tạo ra các đoạn văn bản ngắn hơn (target_sequences_length) để giảm 
                thiểu sựu khác biệt giưa quá trình tiền huấn luyện và tinh chỉnh 
        """
        # Xác định chiều dài mong muốn của câu 
        target_seq_length = max_num_tokens # max  = block_size ( kích thước tối đa câu - special tokens)

        # Nếu sự ngẫu nhiên < 10 % tức 0.1  là xác xuất để tạo ra các chuỗi ngắn hơn  
        # ta khởi tạo số mục tiêu này = 2 - > max_num_tokens 
        if random.random() < SHORT_SEQ_PROB:
            target_seq_length = random.randint(2 , max_num_tokens)
        

        # Xử dụng while kiểm tra điều kiện i < len(document):
        while i < len(document):
            # gán phân đoạn câu  = document [i]
            segment = document[i]
            # thêm phân đoạn này vào bộ đệm lưu chữ phân đoạn bước thời gian hiện tại 
            current_chunk.append(segment)
            # Thêm độ dài cho bộ đệm lưu chữ phân đoạn bước thời gian
            # bằng cách  + độ dài ban đầu với độ dài đoạn câu phân đoạn
            current_length += len(segment)

            # Xây dựng điều kiện khi i = len(document) - 1 
            # hoặc độ dài cau hiện tại >= target_seg_length tức là độ dài vượt chỉ tiêu 
            if  i == len(document) - 1 or current_length >= target_seq_length:
                # khởi tạo một biến a_end là bao nhiêu đoạn từ 'current_chunk 
                # đi vào A từ câu đầu tiên
                if current_chunk : # nếu có tồn tại bộ đệm lưu trữ phân đoạn bược thời gia 
                    # Hiện tại 
                    a_end = 1

                    if len(current_chunk) >= 2 :
                        a_end = random.randint(1 , len(current_chunk) - 1) 
                    
                    tokens_a = []
                    # lặp qua một chuỗi các tokens a_and:
                    for j in range(a_end):
                        # Lặp qua các đoạn văn bản (current chunk từ 0 đến a_end)
                        # với mỗi đoạn văn bản , thêm tất cả các tokens của đoạn đó 
                        # vao a_end sử dụng extend để nối các đoạn danh sách với nhau 
                    # sau khi kết thúc tokens_a sẽ nhận được 1 list 0 -> a_end -1 các 
                    # đoạn tokens của current_chunk 
                        tokens_a.extend(current_chunk[j])

                    # Khởi tạo danh sách token_ b 
                    tokens_b = []
                    #  kiểm tra xem curren_chunk có phải 1 đoạn không 
                    # hoặc xác xuất ngẫu nhiên có nhỏ hơn nsp hay không 
                    # Nếu điều kiện đúng thì tokens_b sẽ được tạo ra từ 1 văn bản ngẫu nhiên khác 
                    # với 1 văn bản hiện tại 
                    if len(current_chunk) == 1 or random.random() < NSP_PROB:
                        is_random_next = True 
                        # sau đó tính toán chiều dài mục tiêu token b 
                        target_b_length = target_seq_length - len(tokens_a)

                        # Khởi tạo biến random_document_index để lưu chủ những chỉ số của văn bản
                        # ngẫu nhiên  lặp 10 lần để chánh chọn lại văn bản hiện tại 
                        for _ in range(10):
                            # Với mỗi lần lặp gán cho random_document_index 
                            # bộ chỉ số idx từ 0 đến danh sách các văn bản được tách thành các câu - 1
                            random_document_index = random.randint(
                                0 , len(examples['tokenized_sentences']) - 1
                            )
                            # và kiểm trả xem các chỉ số indx có khác hay không nếu 0 khác tiêp 
                        
                            if random_document_index != doc_index:
                                break 
                        # sau đó gán random_document bằng phần tử tệp văn bản đã tách câu 
                        # tại những chỉ số tương tự của random_document_indx
                        random_document = examples['tokenized_sentences'][
                            random_document_index 
                        ]
                        # khởi tạo biến random_start  = 0 -> len_radom_document - 1 
                        random_start = random.randint(0 , len(random_document) - 1)
                        # Duyệt qua các câu trong random_document từ chỉ số satrt đến hết 
                        for j in range(random_start , len(random_document)):
                            # Lặp qua các đoạn văn bản random_documen và thêm các chỉ số tokens của câu
                            # và sử dụng extend để nối các câu với nhau 
                            tokens_b.extend(random_document[j])
                            # sau đó kiểm trả xem độ dài tokens b có lớn hoặc băngc hiều đai mục tiêu 
                            # mong muốn cho b hay không 
                            if len(tokens_b) >= target_b_length:
                                    break
                            
                            # sau đó tính số lượng đoạn văn bản không được sử dụng 
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # trường hợp còn lại 
                    else: 
                        is_random_next = False 
                        # duyệt qua danh sách current_chunk và thêm các chỉ số a_end 
                        # cho tokens_b , sử dụng extend để nối các câu với nhau 
                        for j in range(a_end , len(current_chunk)):
                            # sử dụng extend để nối các câu với nhau 
                            tokens_b.extend(current_chunk[j])

                    # Thiết lập đâù vào từ 2 danh sách tokens_a , tokens_b 
                    input_ids = tokenizer.build_inputs_width_special_tokens(
                        tokens_a , tokens_b
                    )
                    # Biến đổi type_tokens_ids 0 for sentences a , 1 for sentences b 
                    tokens_type_ids = tokenizer.create_token_type_ids_from_sequences(
                        tokens_a , tokens_b
                    )
                    # Đệm cho tokens_ids và tokens_tyoe_ids 
                    padded = tokenizer.pad(
                        {"input_ids" :input_ids , " tokens_type_ids": tokens_type_ids},
                        padding = "max_length", 
                        max_length=MAX_LENGTH,
                    )

                    examples['input_ids'].append(padded['input_ids'])
                    examples['token_type_ids'].append(padded['token_type_ids'])
                    examples["attention_mask"].append(padded['attention_mask']) # 1 for non-paded , 0 for padded
                    examples['next_sentences_labels'].append(1 if is_random_next else 0)

            i+ 1 
        
    # Xóa đi tất cả các cột dữ liệu không cần thiết từ tệp dự liệu 
    del  examples['document']
    del examples['sentences']
    del examples['text']
    del examples['tokenized_sentences']

    return examples 

tokenized_dataset = dataset.map(
    prepare_train_features , batchched=True , remove_columns=['text'] , num_proc =1, 
)
                

In [ ]:
from transformers import DataCollatorForLanguageModeling 

collater = DataCollatorForLanguageModeling(
    # tokenizer hóa văn bản , xacs xuất mặt nạ cho mlm = 0.2  20 % return tensor 
    tokeinzer= tokenizer , mlm=True , mlm_probability=MLM_PROB , return_tensors='tf'
)

In [ ]:
train = tokenized_dataset['train'].to_tf_dataset(
    columns= ['input_ids' , 'token_type_ids' , 'attention_mask'],
    label_col = ['labels' , 'next_sentences_label'],
    batch_size = TRAIN_BATCH_SIZE , 
    shuffle = True , 
    cllate_fn = collater , 
)

validation = tokenized_dataset['validation'].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels", "next_sentence_label"],
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    collate_fn=collater,
)

Defining the model 

In [ ]:
from transformers import BertConfig 
config = BertConfig.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
from transformers import TFBertForPreTraining

model = TFBertForPreTraining(config)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer)

In [ ]:
model.fit(train, validation_data=validation, epochs=MAX_EPOCHS)

In [ ]:
model.push_to_hub('pretrained-bert' , organization='keras-io')
tokenizer.push_to_hub('pretrained-bert', organization='keras-io')

from transformers import TFBertForPreTraining 
model = TFBertForPreTraining.from_pretrained('your-username/my-awesome-model')

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("your-username/my-awesome-model")